In [1]:
!pip install google_play_scraper

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from google_play_scraper import app, reviews

In [3]:
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/troyjeffrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/troyjeffrey/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## **Data Collection using Google Play Store API**

In [30]:
# Specify the package name of the Spotify app
app_package = 'com.spotify.music'

# Fetch reviews using the reviews function
reviews_result = reviews(
    app_package,
    lang='en',  # Specify the language of the reviews
    count=50   # Number of reviews to fetch
)

# Print the extracted reviews
print(reviews_result)


([{'reviewId': '2bf344f3-3504-4b7c-aef8-2fcc408f2520', 'userName': 'B K', 'userImage': 'https://play-lh.googleusercontent.com/a/AAcHTtfaI5tlcvOu7O1WoTMx_HEIf57lBnmlEW69ubVUhCV3=mo', 'content': 'The amount of advertising has exploded. This is a great app if you want to listen to two songs, followed by three ads. I am looking for another option, because Spotify has definitely declined recently. I am going back to iheart radio. I cannot believe how many ads I have to listen to - the radio is better than this!', 'score': 1, 'thumbsUpCount': 2, 'reviewCreatedVersion': '8.8.58.473', 'at': datetime.datetime(2023, 8, 8, 14, 2, 5), 'replyContent': 'Hi! We’re sorry to hear this and we’ll pass your feedback along to the Ads team. For an ad free experience, we’d recommend checking out our Premium plans. If you’ve never had Premium before, you should be eligible for our trial. You can also check out other deals by heading over to spotify.com/premium.', 'repliedAt': datetime.datetime(2023, 8, 9, 1, 

In [6]:
reviews_result = []

# Fetch reviews from 5 pages (adjust as needed)
for page_num in range(1, 50):
    if page_num == 1:
        reviews_page = reviews(
            app_package,
            lang='en',      # Specify the language of the reviews
            count=100,      # Number of reviews to fetch per page
            sort='recent'   # Sort by most recent reviews
        )
    else:
        reviews_page = reviews(
            app_package,
            lang='en',
            count=100,
            sort='recent',
            continuation_token=reviews_page[1]
        )
    reviews_result.extend(reviews_page[0])

# Create a DataFrame from the list of reviews data
reviews_df = pd.DataFrame(reviews_result)

# Display the DataFrame
reviews_df[['content', 'score', 'at']].head()

,content,score,at
0,I absolutely love how extensive this app has w...,3,2023-07-31 13:14:37
1,"I don't know what happened, but for ONE day, I...",3,2023-08-03 09:09:28
2,"I don't normally leave reviews, but enough is ...",2,2023-07-23 12:37:42
3,"The ""plus"" update could get confusing when you...",4,2023-08-01 10:24:48
4,Desktop Spotify is decent. You can reorder son...,1,2023-08-03 00:01:03


## **Data Cleaning**

In [75]:
reviews_df.rename(columns={'content': 'review', 'score': 'rating', 'at': 'date'}, inplace=True)

In [8]:
reviews_df.columns

Index(['reviewId', 'userName', 'userImage', 'review', 'rating',
       'thumbsUpCount', 'reviewCreatedVersion', 'date', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')

In [19]:
reviews_data = reviews_df.copy()

In [22]:
# Drop duplicates
reviews_data = reviews_data.drop_duplicates()
# convert date to dattime format 
reviews_data['date'] = pd.to_datetime(reviews_data['date'])
# fill Nan Values 
# for example, fill NaN in 'Review' column with an empty string
reviews_data['review'] = reviews_data['review'].fillna('')
# data type conversions 
reviews_data['rating'] = reviews_data['rating'].astype(int)
# Drop unnecessary columns
columns_to_drop = ['reviewId', 'userName', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'replyContent', 'repliedAt']
reviews_data = reviews_df.drop(columns=columns_to_drop)
reviews_data.head(20)



,review,rating,date,appVersion
0,I absolutely love how extensive this app has w...,3,2023-07-31 13:14:37,8.8.56.538
1,"I don't know what happened, but for ONE day, I...",3,2023-08-03 09:09:28,8.8.56.538
2,"I don't normally leave reviews, but enough is ...",2,2023-07-23 12:37:42,8.8.54.481
3,"The ""plus"" update could get confusing when you...",4,2023-08-01 10:24:48,8.8.56.538
4,Desktop Spotify is decent. You can reorder son...,1,2023-08-03 00:01:03,8.8.56.538
5,"Horrible! This app is awful, I've been using i...",1,2023-07-25 20:36:37,8.8.54.481
6,I am a user who uses the free version and have...,3,2023-08-02 23:29:36,8.8.56.538
7,Please add more options for editing Playlists ...,3,2023-07-26 12:04:16,8.8.54.481
8,I had previously given this a 3 star rating be...,3,2023-07-25 09:53:07,8.8.54.481
9,"Great music app, but there's no reason to leav...",1,2023-07-25 05:43:46,8.8.54.481


In [70]:
reviews_data['rating'].mean()

2.4451020408163266

In [71]:
reviews_data['appVersion'].value_counts()

8.8.36.522    775
8.8.54.481    725
8.8.40.470    687
8.8.52.458    452
8.8.56.538    370
8.8.50.466    352
8.8.48.523    352
8.8.44.527    351
8.8.46.529    249
8.8.42.468    139
8.8.38.444     76
8.8.22.510     65
8.8.36.521     59
8.8.18.509     46
8.8.28.409     43
8.8.20.544     40
8.8.26.408     21
8.8.24.307     19
8.8.32.508     15
8.8.14.575     14
8.8.12.545     14
8.8.16.615     13
8.8.10.582     11
8.8.34.429      5
8.8.58.473      4
8.8.52.2        1
8.8.26.56       1
8.8.62.68       1
Name: appVersion, dtype: int64

In this example, the code defines functions to preprocess the text by converting it to lowercase and removing special characters. It then uses NLTK to tokenize the sentences and words in the cleaned review text. The resulting DataFrame includes columns with the original review, cleaned review, tokenized sentences, and tokenized words.

## **Preprocessing raw text and tokenizing sentences**

In [24]:
# Clean and preprocess the review text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and symbols using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    return text

# Apply preprocessing function to the 'review' column
reviews_data['Cleaned Review'] = reviews_data['review'].apply(preprocess_text)

# Tokenize sentences using NLTK
def tokenize_sentences(text):
    return sent_tokenize(text)

# Tokenize words using NLTK
def tokenize_words(text):
    return word_tokenize(text)

# Apply tokenization functions to the cleaned review text
reviews_data['Sentences'] = reviews_data['Cleaned Review'].apply(tokenize_sentences)
reviews_data['Tokens'] = reviews_data['Cleaned Review'].apply(tokenize_words)

# Display the cleaned and tokenized DataFrame
reviews_data[['review', 'Cleaned Review', 'Sentences', 'Tokens']].head(50)

,review,Cleaned Review,Sentences,Tokens
0,I absolutely love how extensive this app has w...,i absolutely love how extensive this app has w...,[i absolutely love how extensive this app has ...,"[i, absolutely, love, how, extensive, this, ap..."
1,"I don't know what happened, but for ONE day, I...",i dont know what happened but for one day i ha...,[i dont know what happened but for one day i h...,"[i, dont, know, what, happened, but, for, one,..."
2,"I don't normally leave reviews, but enough is ...",i dont normally leave reviews but enough is en...,[i dont normally leave reviews but enough is e...,"[i, dont, normally, leave, reviews, but, enoug..."
3,"The ""plus"" update could get confusing when you...",the plus update could get confusing when you w...,[the plus update could get confusing when you ...,"[the, plus, update, could, get, confusing, whe..."
4,Desktop Spotify is decent. You can reorder son...,desktop spotify is decent you can reorder song...,[desktop spotify is decent you can reorder son...,"[desktop, spotify, is, decent, you, can, reord..."
5,"Horrible! This app is awful, I've been using i...",horrible this app is awful ive been using it f...,[horrible this app is awful ive been using it ...,"[horrible, this, app, is, awful, ive, been, us..."
6,I am a user who uses the free version and have...,i am a user who uses the free version and have...,[i am a user who uses the free version and hav...,"[i, am, a, user, who, uses, the, free, version..."
7,Please add more options for editing Playlists ...,please add more options for editing playlists ...,[please add more options for editing playlists...,"[please, add, more, options, for, editing, pla..."
8,I had previously given this a 3 star rating be...,i had previously given this a 3 star rating be...,[i had previously given this a 3 star rating b...,"[i, had, previously, given, this, a, 3, star, ..."
9,"Great music app, but there's no reason to leav...",great music app but theres no reason to leave ...,[great music app but theres no reason to leave...,"[great, music, app, but, theres, no, reason, t..."


In [ ]:
df.describe()

In this example, the code initializes the VADER Sentiment Intensity Analyzer from NLTK. The analyze_sentiment function uses VADER to calculate sentiment scores and categorizes the sentiment as positive, negative, or neutral based on the compound score. The sentiment analysis results are added to a new column called 'Sentiment' in the DataFrame.

## **VADER Sentiment Analysis**

In [37]:
!pip install textblob


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 4.6 MB/s eta 0:00:00a 0:00:01


In [63]:
reviews_data['sentiment category'].describe()

count        4900
unique          3
top       Neutral
freq         4791
Name: sentiment category, dtype: object

In [73]:
!pip install transformers

In [74]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

/Users/troyjeffrey/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████████████████████████| 687/687 [00:00<00:00, 209kB/s]
Downloading: 100%|█████████████████████████| 1.42G/1.42G [01:11<00:00, 19.9MB/s]
Downloading: 100%|██████████████████████████████| 256/256 [00:00<00:00, 280kB/s]
Downloading: 100%|███████████████████████████| 798k/798k [00:00<00:00, 7.23MB/s]
Downloading: 100%|███████████████████████████| 456k/456k [00:00<00:00, 5.26MB/s]
Downloading: 100%|█████████████████████████████| 150/150 [00:00<00:00, 36.4kB/s]


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [76]:
reviews_data.dtypes

review                        object
rating                         int64
date                  datetime64[ns]
appVersion                    object
Cleaned Review                object
Sentences                     object
Tokens                        object
Sentiment Score              float64
Sentiment                     object
Sentiment Category            object
sentiment                    float64
sentiment Category            object
adjusted sentiment           float64
sentiment category            object
dtype: object

In [77]:
reviews_data['review'] = reviews_data['review'].astype('str')

In [78]:
reviews_data['result'] = reviews_data['review'].apply(lambda x: sentiment_analysis(x))

In [81]:
reviews_data.head(10)

,review,rating,date,appVersion,Cleaned Review,Sentences,Tokens,Sentiment Score,Sentiment,Sentiment Category,sentiment,sentiment Category,adjusted sentiment,sentiment category,result
0,"I absolutely love how extensive this app has when it comes to music. There is amazing music and they even have unknown artists that can be discovered. But recently, I was listening to music while I was taking a shower with the free version and I had gotten an ad after every song. It was very annoying, and another thing I have a problem with is when I connect to my alexa, the 30 minute ad free option just doesnt work. It is very confusing and frustrating.",3,2023-07-31 13:14:37,8.8.56.538,i absolutely love how extensive this app has when it comes to music there is amazing music and they even have unknown artists that can be discovered but recently i was listening to music while i was taking a shower with the free version and i had gotten an ad after every song it was very annoying and another thing i have a problem with is when i connect to my alexa the 30 minute ad free option just doesnt work it is very confusing and frustrating,[i absolutely love how extensive this app has when it comes to music there is amazing music and they even have unknown artists that can be discovered but recently i was listening to music while i was taking a shower with the free version and i had gotten an ad after every song it was very annoying and another thing i have a problem with is when i connect to my alexa the 30 minute ad free option just doesnt work it is very confusing and frustrating],"[i, absolutely, love, how, extensive, this, app, has, when, it, comes, to, music, there, is, amazing, music, and, they, even, have, unknown, artists, that, can, be, discovered, but, recently, i, was, listening, to, music, while, i, was, taking, a, shower, with, the, free, version, and, i, had, gotten, an, ad, after, every, song, it, was, very, annoying, and, another, thing, i, have, a, problem, with, is, when, i, connect, to, my, alexa, the, 30, minute, ad, free, option, just, doesnt, work, it, is, very, confusing, and, frustrating]",-0.1351,Negative,Positive,0.001000,Negative,0.000600,Negative,"[{'label': 'NEGATIVE', 'score': 0.9994747042655945}]"
1,"I don't know what happened, but for ONE day, I had a really cool feature that let me add a song to multiple playlists at once using individual checkboxes on the Add To Playlist screen. Not sure where that feature went, but it's gone from my app now. Checked for an update, but there isn't one. If I'm missing something, PLEASE HELP! I want that fast-add feature back!",3,2023-08-03 09:09:28,8.8.56.538,i dont know what happened but for one day i had a really cool feature that let me add a song to multiple playlists at once using individual checkboxes on the add to playlist screen not sure where that feature went but its gone from my app now checked for an update but there isnt one if im missing something please help i want that fastadd feature back,[i dont know what happened but for one day i had a really cool feature that let me add a song to multiple playlists at once using individual checkboxes on the add to playlist screen not sure where that feature went but its gone from my app now checked for an update but there isnt one if im missing something please help i want that fastadd feature back],"[i, dont, know, what, happened, but, for, one, day, i, had, a, really, cool, feature, that, let, me, add, a, song, to, multiple, playlists, at, once, using, individual, checkboxes, on, the, add, to, playlist, screen, not, sure, where, that, feature, went, but, its, gone, from, my, app, now, checked, for, an, update, but, there, isnt, one, if, im, missing, something, please, help, i, want, that, fastadd, feature, back]",0.7267,Positive,Negative,-0.025000,Negative,-0.015000,Negative,"[{'label': 'NEGATIVE', 'score': 0.9976861476898193}]"
2,"I don't normally leave reviews, but enough is enough. I love spotify, and using it for background musi

In [1]:
reviews_data['result'[0]]

NameError: name 'reviews_data' is not defined